In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import glob
import numpy as np
import cv2 as cv
import argparse
import albumentations
from model import model
from utils.helpers import draw_test_segmentation_map, image_overlay
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [2]:
transform = albumentations.Compose([
    albumentations.Normalize(
            mean=[0.45734706, 0.43338275, 0.40058118],
            std=[0.23965294, 0.23532275, 0.2398498],
            always_apply=True)
])

In [3]:
DEVICE = 'cuda:0'
CLASSES_TO_TRAIN = ['bg', 'wet_road_region', 'road_region', 'mud', 'earthen_patch', 'mountain-stones', 'dirt', 'vegitation_misc', 'distressed_patch', 'drainage_grate', 'water_puddle', 'speed_breaker', 'misc', 'gravel_patch', 'concrete_material']

In [4]:
model = model
checkpoint = torch.load('/home/adnan/Atif/Misc/Anomaly-Seg/Code/model_11.pth', map_location='cuda:0')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval().to(DEVICE)
criterion = nn.CrossEntropyLoss()

In [21]:
root = "/home/adnan/Atif/Misc/Anomaly-Seg/Code/TT/I"
dim = (256, 144)

data = glob.glob(f'{root}/*.jpg')
count = len(data)
for i in tqdm(range(count), desc='Iterations completed'):
    name = data[i].split('I/')[1]
    image = cv.resize(cv.imread(data[i]), dim)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    orig_image = image
    image = transform(image=image)['image']
    image = np.transpose(image, (2, 0, 1))
    image = torch.tensor(image, dtype=torch.float)
    image = image.unsqueeze(0).to(DEVICE)

    outputs = model(image)
    outputs = outputs['out']
    segmented_image = draw_test_segmentation_map(outputs)
    
    map_ = cv.cvtColor(segmented_image, cv.COLOR_BGR2RGB)
    result = image_overlay(orig_image, map_)
    
    cv.imwrite(f"{root}/{name}___model_11.png", result)

Iterations completed: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 44.33it/s]
